# Logic gates using RNA editing

This notebook uses [Julia](https://julialang.org)

In [ ]:
using Catalyst #https://github.com/SciML/Catalyst.jl
using DifferentialEquations #https://github.com/SciML/DifferentialEquations.jl
using Plots #https://github.com/JuliaPlots/Plots.jl

## A buffer gate using a single PPR editing factor to create a start codon

<img src="img/buffer_gate.png">

In [ ]:
rn = @reaction_network begin
    transcriptionrate, DNA --> DNA + ACG
    PPRonrate, ACG + PPR --> ACG_PPR
    PPRoffrate, ACG_PPR --> ACG + PPR
    PPRonrate, AUG + PPR --> AUG_PPR
    PPRoffrate, AUG_PPR --> AUG + PPR
    editingrate, ACG_PPR --> AUG_PPR
    ribosomeonrate, AUG + R --> AUG_R
    ribosomeAUGoffrate, AUG_R --> AUG + R
    AUGtranslationrate, AUG_R --> GFP + AUG + R
    ribosomeonrate, ACG + R --> ACG_R
    ribosomeACGoffrate, ACG_R --> ACG + R
    ACGtranslationrate, ACG_R --> GFP + ACG + R
    RNAdegradationrate, ACG --> ∅
    RNAdegradationrate, AUG --> ∅
    GFPdegradationrate, GFP --> ∅
end

In [ ]:
osys = convert(ODESystem, rn)

In [ ]:
#adjustable parameters
PPRoffrate = 1e-3
ribosomeAUGoffrate = 1e-3

#fixed parameters
transcriptionrate = 1e-2
PPRonrate = 1e-2
editingrate = 0.1
ribosomeonrate = 1e-2
AUGtranslationrate = 0.1
ribosomeACGoffrate = 10 * ribosomeAUGoffrate
ACGtranslationrate = 1e-4
RNAdegradationrate = 5e-4
GFPdegradationrate = 5e-4

pmap = Pair.(parameters(rn), [transcriptionrate, PPRonrate, PPRoffrate, editingrate,
       ribosomeonrate, ribosomeAUGoffrate, AUGtranslationrate, ribosomeACGoffrate, ACGtranslationrate,
        RNAdegradationrate, GFPdegradationrate])

In [ ]:
# time interval to solve on
tspan = (0.0, 10000.0)

#adjustable parameter
numPPR = 100

#initial conditions at t=0
u₀map  = Pair.(species(rn), [100,0,numPPR,0,0,0,100,0,0,0])

In [ ]:
op = ODEProblem(osys, u₀map, tspan, pmap)
sol   = solve(op, AutoTsit5(Rosenbrock23()));     # Tsit5 and Rosenbrock23 are ODE solvers

In [ ]:
plot(sol, lw=2, xguide="time (seconds)", yguide="number of molecules")

In [ ]:
plot(sol.t, [x[9] for x in sol.u], lw=2, xguide="time (seconds)", yguide="number of molecules", label="GFP", color=:limegreen)

## A NOT gate using a single PPR editing factor to create a stop codon

<img src="img/NOT_gate.png">

In [ ]:
rn = @reaction_network begin
    transcriptionrate, DNA --> DNA + CAA
    PPRonrate, CAA + PPR --> CAA_PPR
    PPRoffrate, CAA_PPR --> CAA + PPR
    PPRonrate, UAA + PPR --> UAA_PPR
    PPRoffrate, UAA_PPR --> UAA + PPR
    editingrate, CAA_PPR --> UAA_PPR
    ribosomeonrate, UAA + R --> UAA_R
    ribosomeUAAoffrate, UAA_R --> UAA + R
    ribosomeonrate, CAA + R --> CAA_R
    ribosomeCAAoffrate, CAA_R --> CAA + R
    CAAtranslationrate, CAA_R --> GFP + CAA + R
    RNAdegradationrate, CAA --> ∅
    RNAdegradationrate, UAA --> ∅
    GFPdegradationrate, GFP --> ∅
end
osys = convert(ODESystem, rn);

In [ ]:
#adjustable parameters
PPRoffrate = 1e-3
ribosomeCAAoffrate = 1e-3

#fixed parameters
transcriptionrate = 1e-2
PPRonrate = 1e-2
editingrate = 0.1
ribosomeonrate = 1e-2
CAAtranslationrate = 0.1
ribosomeUAAoffrate = 10 * ribosomeCAAoffrate
RNAdegradationrate = 5e-4
GFPdegradationrate = 5e-4

pmap = Pair.(parameters(rn), [transcriptionrate, PPRonrate, PPRoffrate, editingrate,
        ribosomeonrate, ribosomeCAAoffrate, CAAtranslationrate, ribosomeUAAoffrate,
        RNAdegradationrate, GFPdegradationrate])

In [ ]:
# time interval to solve on
tspan = (0.0, 10000.0)

#adjustable parameter
numPPR = 100

#initial conditions at t=0
u₀map  = Pair.(species(rn), [100, 0, numPPR, 0, 0, 0, 100, 0, 0, 10000])

In [ ]:
op = ODEProblem(osys, u₀map, tspan, pmap)
sol   = solve(op, AutoTsit5(Rosenbrock23()));     # use Tsit5 or Rosenbrock23 ODE solver

In [ ]:
plot(sol.t, [x[10] for x in sol.u], lw=2, legend = false, xguide="time (seconds)", yguide="number of molecules",label="GFP", color=:limegreen)

## An OR gate using two PPR editing factors to edit alternative start codons

<img src="img/OR_gate.png">

In [ ]:
rn = @reaction_network begin
    transcriptionrate, DNA --> DNA + ACGACG
    PPRonrate, ACGACG + PPR1 --> ACGACG_PPR1
    PPR1offrate, ACGACG_PPR1 --> ACGACG + PPR1
    PPRonrate, ACGACG + PPR2 --> ACGACG_PPR2
    PPR2offrate, ACGACG_PPR2 --> ACGACG + PPR2
    PPRonrate, AUGACG + PPR1 --> AUGACG_PPR1
    PPR1offrate, AUGACG_PPR1 --> AUGACG + PPR1
    PPRonrate, AUGACG + PPR2 --> AUGACG_PPR2
    PPR2offrate, AUGACG_PPR2 --> AUGACG + PPR2
    PPRonrate, ACGAUG + PPR1 --> ACGAUG_PPR1
    PPR1offrate, ACGAUG_PPR1 --> ACGAUG + PPR1
    PPRonrate, ACGAUG + PPR2 --> ACGAUG_PPR2
    PPR2offrate, ACGAUG_PPR2 --> ACGAUG + PPR2
    PPRonrate, AUGAUG + PPR1 --> AUGAUG_PPR1
    PPR1offrate, AUGAUG_PPR1 --> AUGAUG + PPR1
    PPRonrate, AUGAUG + PPR2 --> AUGAUG_PPR2
    PPR2offrate, AUGAUG_PPR2 --> AUGAUG + PPR2
    editingrate, ACGACG_PPR1 --> AUGACG_PPR1
    editingrate, ACGACG_PPR2 --> ACGAUG_PPR2
    ribosomeonrate, ACGACG + R --> ACGACG_R
    ribosomeACGoffrate, ACGACG_R --> ACGACG + R
    ACGtranslationrate, ACGACG_R --> GFP + ACGACG + R
    ribosomeonrate, AUGACG + R --> AUGACG_R
    ribosomeAUGoffrate, AUGACG_R --> AUGACG + R
    AUGtranslationrate, AUGACG_R --> GFP + AUGACG + R
    ribosomeonrate, ACGAUG + R --> ACGAUG_R
    ribosomeAUGoffrate, ACGAUG_R --> ACGAUG + R
    AUGtranslationrate, ACGAUG_R --> GFP + ACGAUG + R
    ribosomeonrate, AUGAUG + R --> AUGAUG_R
    ribosomeAUGoffrate, AUGAUG_R --> AUGAUG + R
    AUGtranslationrate, AUGAUG_R --> GFP + AUGAUG + R
    RNAdegradationrate, ACGACG --> ∅
    RNAdegradationrate, ACGAUG --> ∅
    RNAdegradationrate, AUGACG --> ∅
    RNAdegradationrate, AUGAUG --> ∅
    GFPdegradationrate, GFP --> ∅
end
osys = convert(ODESystem, rn);

The ideal result should look like this:

In [ ]:
heatmap([[0,1] [1,1]], c = palette(:linear_green_5_95_c69_n256), ratio = :equal, xlims=(1,2), ylims=(1,2),
    size=(200,200), xticks=[], yticks=[], xlabel="[PPR1]", ylabel="[PPR2]")

In [ ]:
#adjustable parameters
PPR1offrate = 1e-3
PPR2offrate = 1e-3
ribosomeAUGoffrate = 1e-3

#fixed parameters
transcriptionrate = 1e-2
PPRonrate = 1e-2
editingrate = 0.1
ribosomeonrate = 1e-2
AUGtranslationrate = 0.1
ribosomeACGoffrate = 10 * ribosomeAUGoffrate
ACGtranslationrate = 1e-4
RNAdegradationrate = 5e-4
GFPdegradationrate = 5e-4

p = (transcriptionrate, PPRonrate, PPR1offrate, PPR2offrate, editingrate,
        ribosomeonrate, ribosomeACGoffrate, ACGtranslationrate, ribosomeAUGoffrate, AUGtranslationrate,
        RNAdegradationrate, GFPdegradationrate);

In [ ]:
#initial conditions at t=0
u₀ = [100., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 100., 0., 0., 0., 0., 0.]
steps = 20
results = zeros(Float64,steps,steps)
dynamic_range = exp10.(range(0, stop=3, length=steps)) # 1 - 1000 molecules per 'cell'
for (i, amount1) in enumerate(dynamic_range), (j, amount2) in enumerate(dynamic_range)
    u₀[3] = amount1
    u₀[5] = amount2
    u₀map  = Pair.(species(rn), u₀)
    ssprob = SteadyStateProblem(rn, u₀, p)
    sssol  = solve(ssprob, DynamicSS(AutoTsit5(Rosenbrock23())))
    results[i,j] = sssol.u[18]
end

In [ ]:
heatmap(results, c = palette(:linear_green_5_95_c69_n256), ratio = :equal, 
    xlims=(0.5,steps+0.5), ylims=(0.5,steps+0.5),
    xticks=[], yticks=[], xlabel="[PPR1]", ylabel="[PPR2]")

## A NOR gate using two PPR editing factors to edit alternative stop codons

<img src="img/NOR_gate.png">

In [ ]:
rn = @reaction_network begin
    transcriptionrate, DNA --> DNA + CAACAA
    PPRonrate, CAACAA + PPR1 --> CAACAA_PPR1
    PPR1offrate, CAACAA_PPR1 --> CAACAA + PPR1
    PPRonrate, CAACAA + PPR2 --> CAACAA_PPR2
    PPR2offrate, CAACAA_PPR2 --> CAACAA + PPR2
    PPRonrate, UAACAA + PPR1 --> UAACAA_PPR1
    PPR1offrate, UAACAA_PPR1 --> UAACAA + PPR1
    PPRonrate, UAACAA + PPR2 --> UAACAA_PPR2
    PPR2offrate, UAACAA_PPR2 --> UAACAA + PPR2
    PPRonrate, CAAUAA + PPR1 --> CAAUAA_PPR1
    PPR1offrate, CAAUAA_PPR1 --> CAAUAA + PPR1
    PPRonrate, CAAUAA + PPR2 --> CAAUAA_PPR2
    PPR2offrate, CAAUAA_PPR2 --> CAAUAA + PPR2
    PPRonrate, UAAUAA + PPR1 --> UAAUAA_PPR1
    PPR1offrate, UAAUAA_PPR1 --> UAAUAA + PPR1
    PPRonrate, UAAUAA + PPR2 --> UAAUAA_PPR2
    PPR2offrate, UAAUAA_PPR2 --> UAAUAA + PPR2
    editingrate, CAACAA_PPR1 --> UAACAA_PPR1
    editingrate, CAACAA_PPR2 --> CAAUAA_PPR2
    ribosomeonrate, CAACAA + R --> CAACAA_R
    ribosomeCAAoffrate, CAACAA_R --> CAACAA + R
    CAAtranslationrate, CAACAA_R --> GFP + CAACAA + R
    ribosomeonrate, UAACAA + R --> UAACAA_R
    ribosomeUAAoffrate, UAACAA_R --> UAACAA + R
    ribosomeonrate, CAAUAA + R --> CAAUAA_R
    ribosomeUAAoffrate, CAAUAA_R --> CAAUAA + R
    ribosomeonrate, UAAUAA + R --> UAAUAA_R
    ribosomeUAAoffrate, UAAUAA_R --> UAAUAA + R
    RNAdegradationrate, CAACAA --> ∅
    RNAdegradationrate, CAAUAA --> ∅
    RNAdegradationrate, UAACAA --> ∅
    RNAdegradationrate, UAAUAA --> ∅
    GFPdegradationrate, GFP --> ∅
end
osys = convert(ODESystem, rn);

The ideal result should look like this:

In [ ]:
heatmap([[1,0] [0,0]], c = palette(:linear_green_5_95_c69_n256), ratio = :equal, xlims=(1,2), ylims=(1,2),
    size=(200,200), xticks=[], yticks=[], xlabel="[PPR1]", ylabel="[PPR2]")

In [ ]:
#adjustable parameters
PPR1offrate = 1e-3
PPR2offrate = 1e-3
ribosomeCAAoffrate = 1e-3

#fixed parameters
transcriptionrate = 1e-2
PPRonrate = 1e-2
editingrate = 0.1
ribosomeonrate = 1e-2
CAAtranslationrate = 0.1
ribosomeUAAoffrate = 10 * ribosomeCAAoffrate
RNAdegradationrate = 5e-4
GFPdegradationrate = 5e-4

p = (transcriptionrate, PPRonrate, PPR1offrate, PPR2offrate, editingrate,
        ribosomeonrate, ribosomeCAAoffrate, CAAtranslationrate, ribosomeUAAoffrate,
        RNAdegradationrate, GFPdegradationrate);

In [ ]:
#initial conditions at t=0
u₀ = [100., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 100., 0., 0., 0., 0., 0.]
steps = 20
results = zeros(Float64,steps,steps)
dynamic_range = exp10.(range(0, stop=3, length=steps)) # 1 - 1000 molecules per 'cell'
for (i, amount1) in enumerate(dynamic_range), (j, amount2) in enumerate(dynamic_range)
    u₀[3] = amount1
    u₀[5] = amount2
    u₀map  = Pair.(species(rn), u₀)
    ssprob = SteadyStateProblem(rn, u₀, p)
    sssol  = solve(ssprob, DynamicSS(AutoTsit5(Rosenbrock23())))
    results[i,j] = sssol.u[18]
end

In [ ]:
heatmap(results, c = palette(:linear_green_5_95_c69_n256), ratio = :equal, 
    xlims=(0.5,steps+0.5), ylims=(0.5,steps+0.5),
    xticks=[], yticks=[], xlabel="[PPR1]", ylabel="[PPR2]")

## An AND gate using two PPR editing factors: a C->U editing factor to create a start codon, and a U->C editing factor to remove a stop codon

<img src="img/AND_gate.png">

In [ ]:
rn = @reaction_network begin
    transcriptionrate, DNA --> DNA + ACGUAA
    PPRonrate, ACGUAA + PPR1 --> ACGUAA_PPR1
    PPRonrate, ACGUAA + PPR2 --> ACGUAA_PPR2
    PPRonrate, AUGUAA + PPR1 --> AUGUAA_PPR1
    PPRonrate, AUGUAA + PPR2 --> AUGUAA_PPR2
    PPRonrate, ACGCAA + PPR1 --> ACGCAA_PPR1
    PPRonrate, ACGCAA + PPR2 --> ACGCAA_PPR2
    PPRonrate, AUGCAA + PPR1 --> AUGCAA_PPR1
    PPRonrate, AUGCAA + PPR2 --> AUGCAA_PPR2
    PPR1offrate, ACGUAA_PPR1 --> ACGUAA + PPR1
    PPR2offrate, ACGUAA_PPR2 --> ACGUAA + PPR2
    PPR1offrate, AUGUAA_PPR1 --> AUGUAA + PPR1
    PPR2offrate, AUGUAA_PPR2 --> AUGUAA + PPR2
    PPR1offrate, ACGCAA_PPR1 --> ACGCAA + PPR1
    PPR2offrate, ACGCAA_PPR2 --> ACGCAA + PPR2
    PPR1offrate, AUGCAA_PPR1 --> AUGCAA + PPR1
    PPR2offrate, AUGCAA_PPR2 --> AUGCAA + PPR2
    editingrate, ACGUAA_PPR1 --> AUGUAA_PPR1
    editingrate, ACGCAA_PPR1 --> AUGCAA_PPR1
    editingrate, ACGUAA_PPR2 --> ACGCAA_PPR2
    editingrate, AUGUAA_PPR2 --> AUGCAA_PPR2
    ribosomeonrate, ACGUAA + R --> ACGUAA_R
    ribosomeonrate, ACGCAA + R --> ACGCAA_R
    ribosomeonrate, AUGUAA + R --> AUGUAA_R
    ribosomeonrate, AUGCAA + R --> AUGCAA_R
    ribosomeUAAoffrate, ACGUAA_R --> ACGUAA + R
    ribosomeUAAoffrate, AUGUAA_R --> AUGUAA + R
    ribosomeACGoffrate, ACGCAA_R --> ACGCAA + R
    ribosomeAUGoffrate, AUGCAA_R --> AUGCAA + R
    ACGtranslationrate, ACGCAA_R --> GFP + ACGCAA + R
    AUGtranslationrate, AUGCAA_R --> GFP + AUGCAA + R
    RNAdegradationrate, ACGCAA --> ∅
    RNAdegradationrate, ACGUAA --> ∅
    RNAdegradationrate, AUGUAA --> ∅
    RNAdegradationrate, AUGCAA --> ∅
    GFPdegradationrate, GFP --> ∅
end
osys = convert(ODESystem, rn);

The ideal result should look like this:

In [ ]:
heatmap([[0,0] [0,1]], c = palette(:linear_green_5_95_c69_n256), ratio = :equal, xlims=(1,2), ylims=(1,2),
    size=(200,200), xticks=[], yticks=[], xlabel="[PPR1]", ylabel="[PPR2]")

In [ ]:
#adjustable parameters
PPR1offrate = 1e-3
PPR2offrate = 1e-3
ribosomeAUGoffrate = ribosomeCAAoffrate = 1e-3

#fixed parameters
transcriptionrate = 1e-2
PPRonrate = 1e-2
editingrate = 0.1
ribosomeonrate = 1e-2
AUGtranslationrate = 0.1
ribosomeACGoffrate = 10 * ribosomeAUGoffrate
ribosomeUAAoffrate = 10 * ribosomeCAAoffrate
ACGtranslationrate = 1e-4
RNAdegradationrate = 5e-4
GFPdegradationrate = 5e-4

p = (transcriptionrate, PPRonrate, PPR1offrate, PPR2offrate, editingrate,
        ribosomeonrate, ribosomeUAAoffrate, ribosomeACGoffrate, ribosomeAUGoffrate, ACGtranslationrate, AUGtranslationrate,
        RNAdegradationrate, GFPdegradationrate);

In [ ]:
#initial conditions at t=0
u₀ = zeros(Float64, length(species(rn)))
#set DNA to 100
u₀[1] = 100
#set ribosomes to 100
u₀[16] = 100
steps = 20
results = zeros(Float64,steps,steps)
dynamic_range = exp10.(range(0, stop=3, length=steps)) # 1 - 1000 molecules per 'cell'
for (i, amount1) in enumerate(dynamic_range), (j, amount2) in enumerate(dynamic_range)
    #PPR1
    u₀[3] = amount1
    #PPR2
    u₀[5] = amount2
    ssprob = SteadyStateProblem(rn, u₀, p)
    sssol  = solve(ssprob, DynamicSS(AutoTsit5(Rosenbrock23())))
    #GFP
    results[i,j] = sssol.u[21]
end

In [ ]:
heatmap(results, c = palette(:linear_green_5_95_c69_n256), ratio = :equal, 
    xlims=(0.5,steps+0.5), ylims=(0.5,steps+0.5),
    xticks=[], yticks=[], xguide="[PPR1]", yguide="[PPR2]")

## A NAND gate using two PPR U->C editing factors to remove adjacent start codons

<img src="img/NAND_gate.png">

In [ ]:
rn = @reaction_network begin
    transcriptionrate, DNA --> DNA + AUGAUG
    PPRonrate, AUGAUG + PPR1 --> AUGAUG_PPR1
    PPRonrate, AUGAUG + PPR2 --> AUGAUG_PPR2
    PPRonrate, ACGAUG + PPR1 --> ACGAUG_PPR1
    PPRonrate, ACGAUG + PPR2 --> ACGAUG_PPR2
    PPRonrate, AUGACG + PPR1 --> AUGACG_PPR1
    PPRonrate, AUGACG + PPR2 --> AUGACG_PPR2
    PPRonrate, ACGACG + PPR1 --> ACGACG_PPR1
    PPRonrate, ACGACG + PPR2 --> ACGACG_PPR2
    PPR1offrate, ACGAUG_PPR1 --> ACGAUG + PPR1
    PPR2offrate, ACGAUG_PPR2 --> ACGAUG + PPR2
    PPR1offrate, AUGAUG_PPR1 --> AUGAUG + PPR1
    PPR2offrate, AUGAUG_PPR2 --> AUGAUG + PPR2
    PPR1offrate, ACGACG_PPR1 --> ACGACG + PPR1
    PPR2offrate, ACGACG_PPR2 --> ACGACG + PPR2
    PPR1offrate, AUGACG_PPR1 --> AUGACG + PPR1
    PPR2offrate, AUGACG_PPR2 --> AUGACG + PPR2
    editingrate, AUGAUG_PPR1 --> ACGAUG_PPR1
    editingrate, AUGACG_PPR1 --> ACGACG_PPR1
    editingrate, ACGAUG_PPR2 --> ACGACG_PPR2
    editingrate, AUGAUG_PPR2 --> AUGACG_PPR2
    ribosomeonrate, ACGAUG + R --> ACGAUG_R
    ribosomeonrate, ACGACG + R --> ACGACG_R
    ribosomeonrate, AUGAUG + R --> AUGAUG_R
    ribosomeonrate, AUGACG + R --> AUGACG_R
    ribosomeACGoffrate, ACGACG_R --> ACGACG + R
    ribosomeAUGoffrate, AUGACG_R --> AUGACG + R
    ribosomeAUGoffrate, AUGAUG_R --> AUGAUG + R
    ribosomeAUGoffrate, ACGAUG_R --> ACGAUG + R
    ACGtranslationrate, ACGACG_R --> GFP + ACGACG + R
    AUGtranslationrate, AUGACG_R --> GFP + AUGACG + R
    AUGtranslationrate, ACGAUG_R --> GFP + ACGAUG + R
    AUGtranslationrate, AUGAUG_R --> GFP + AUGAUG + R
    RNAdegradationrate, ACGACG --> ∅
    RNAdegradationrate, ACGAUG --> ∅
    RNAdegradationrate, AUGAUG --> ∅
    RNAdegradationrate, AUGACG --> ∅
    GFPdegradationrate, GFP --> ∅
end
osys = convert(ODESystem, rn);

In [ ]:
#adjustable parameters
PPR1offrate = 1e-3
PPR2offrate = 1e-3
ribosomeAUGoffrate = 1e-3

#fixed parameters
transcriptionrate = 1e-2
PPRonrate = 1e-2
editingrate = 0.1
ribosomeonrate = 1e-2
AUGtranslationrate = 0.1
ribosomeACGoffrate = 10 * ribosomeAUGoffrate
ACGtranslationrate = 0
RNAdegradationrate = 5e-4
GFPdegradationrate = 5e-4

p = (transcriptionrate, PPRonrate, PPR1offrate, PPR2offrate, editingrate,
        ribosomeonrate, ribosomeACGoffrate, ribosomeAUGoffrate, ACGtranslationrate, AUGtranslationrate,
        RNAdegradationrate, GFPdegradationrate);

The ideal result should look like this:

In [ ]:
heatmap([[1,1] [1,0]], c = palette(:linear_green_5_95_c69_n256), ratio = :equal, xlims=(1,2), ylims=(1,2),
    size=(200,200), xticks=[], yticks=[], xlabel="[PPR1]", ylabel="[PPR2]")

In [ ]:
#initial conditions at t=0
u₀ = zeros(Float64, length(species(rn)))
#set DNA to 100
u₀[1] = 100
#set ribosomes to 100
u₀[16] = 100
steps = 20
results = zeros(Float64,steps,steps)
dynamic_range = exp10.(range(0, stop=3, length=steps)) # 1 - 1000 molecules per 'cell'
for (i, amount1) in enumerate(dynamic_range), (j, amount2) in enumerate(dynamic_range)
    #PPR1
    u₀[3] = amount1
    #PPR2
    u₀[5] = amount2
    ssprob = SteadyStateProblem(rn, u₀, p)
    sssol  = solve(ssprob, DynamicSS(AutoTsit5(Rosenbrock23())))
    #GFP
    results[i,j] = sssol.u[21]
end

In [ ]:
heatmap(results, c = palette(:linear_green_5_95_c69_n256), ratio = :equal, 
    xlims=(0.5,steps+0.5), ylims=(0.5,steps+0.5),
    xticks=[], yticks=[], xguide="[PPR1]", yguide="[PPR2]")